## Что первое пришло в голову ##

Блин, не применить tf/idf, ngram и word2vec (хотя про последнее не факт, но я ещё не придумал как)

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек слово.

Дальше можно будет проверить разные классификаторы.

In [38]:
import numpy as np
import pandas as pd

# Длина максимального слова (для нормализации), подробнее в 'Testing.ipynb'
MAX_WORD = 57

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩbcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXZ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯaeiouyAEIOUY'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюяaeiouyAEIOUYbcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXZ'

In [39]:
train_companies = open('../kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('../kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

In [40]:
import re
               
def tokenize(company, spaces=False):
    with_spaces = list(filter(None, re.split('(\w+| )', company)))
    if(spaces):
        return with_spaces
    return [token for token in with_spaces if token != ' ']

def has_letters(string):
    return any(char in ALPHABET for char in string)

def has_not_letters(string):
    return any(char not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

В данной строке артефакт, подробнее в 'Analysis.ipynb'

In [41]:
del train_companies[471970]

In [42]:
def extract_features(company, test=False):
    
    # Длина слова, количество гласных/согласных, положение в слове,
    # есть ли в слове что-то помимо букв и то же самое для предыдущего слова
    num_of_features = 21
    
    # 3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in tokenize(company):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, num_of_features))
    labels = np.zeros(size)
    
    index = 0
    tokens = tokenize(company)
    for i, word in enumerate(tokens):
        if(has_letters(word)):
            
            if(index == 0):
                features[index][7] = index/size
                features[index][8] = len(word)/MAX_WORD
                features[index][9] = count_consonants(word)/len(word)
                features[index][10] = count_vowels(word)/len(word)
                features[index][11] = int(has_not_letters(word))
                if(i > 0):
                    features[index][12] = int(tokens[i - 1] == '"')
                    features[index][13] = int(tokens[i - 1] == '(')
                
            else:
                features[index - 1][14] = index/size
                features[index - 1][15] = len(word)/MAX_WORD
                features[index - 1][16] = count_consonants(word)/len(word)
                features[index - 1][17] = count_vowels(word)/len(word)
                features[index - 1][18] = int(has_not_letters(word))
                features[index - 1][19] = int(tokens[i - 1] == '"')
                features[index - 1][20] = int(tokens[i - 1] == '(')
                
                features[index][:14] = features[index - 1][7:]
            
            if(word[0].isupper()):
                if(word.isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return list(features), list(labels)
    else:
        return list(features)

In [43]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features += curr_features
            labels += curr_labels
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

In [44]:
def transform(company, labels):
    word_index = 0
    corrected = tokenize(company, spaces=True)
    for i, word in enumerate(corrected):
        if (has_letters(word)):
            if (labels[word_index] == 0):
                corrected[i] = word.lower()
            elif(labels[word_index] == 1):
                corrected[i] = word[0] + word[1:].lower()
            word_index += 1
    return corrected

In [45]:
X_train, y_train = generate_dataset(train_companies[:100000])

In [46]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier()
clf.fit(X_train, y_train)

Learning rate set to 0.109943
0:	learn: 0.9699590	total: 197ms	remaining: 3m 17s
1:	learn: 0.8722542	total: 362ms	remaining: 3m
2:	learn: 0.7973345	total: 539ms	remaining: 2m 58s
3:	learn: 0.7376447	total: 704ms	remaining: 2m 55s
4:	learn: 0.6869165	total: 892ms	remaining: 2m 57s
5:	learn: 0.6448756	total: 1.07s	remaining: 2m 57s
6:	learn: 0.6094344	total: 1.25s	remaining: 2m 57s
7:	learn: 0.5786612	total: 1.41s	remaining: 2m 55s
8:	learn: 0.5532471	total: 1.6s	remaining: 2m 56s
9:	learn: 0.5297233	total: 1.76s	remaining: 2m 54s
10:	learn: 0.5098813	total: 1.94s	remaining: 2m 54s
11:	learn: 0.4928211	total: 2.1s	remaining: 2m 53s
12:	learn: 0.4755027	total: 2.28s	remaining: 2m 52s
13:	learn: 0.4622535	total: 2.43s	remaining: 2m 51s
14:	learn: 0.4479552	total: 2.63s	remaining: 2m 52s
15:	learn: 0.4353773	total: 2.81s	remaining: 2m 52s
16:	learn: 0.4253333	total: 2.98s	remaining: 2m 52s
17:	learn: 0.4151823	total: 3.21s	remaining: 2m 55s
18:	learn: 0.4061827	total: 3.45s	remaining: 2m 57

158:	learn: 0.2392435	total: 28.1s	remaining: 2m 28s
159:	learn: 0.2389130	total: 28.2s	remaining: 2m 28s
160:	learn: 0.2385442	total: 28.4s	remaining: 2m 27s
161:	learn: 0.2383682	total: 28.6s	remaining: 2m 27s
162:	learn: 0.2380650	total: 28.7s	remaining: 2m 27s
163:	learn: 0.2377228	total: 28.9s	remaining: 2m 27s
164:	learn: 0.2375095	total: 29.1s	remaining: 2m 27s
165:	learn: 0.2372880	total: 29.3s	remaining: 2m 27s
166:	learn: 0.2370602	total: 29.5s	remaining: 2m 27s
167:	learn: 0.2367905	total: 29.6s	remaining: 2m 26s
168:	learn: 0.2366052	total: 29.8s	remaining: 2m 26s
169:	learn: 0.2363879	total: 30s	remaining: 2m 26s
170:	learn: 0.2361296	total: 30.2s	remaining: 2m 26s
171:	learn: 0.2358684	total: 30.4s	remaining: 2m 26s
172:	learn: 0.2355490	total: 30.6s	remaining: 2m 26s
173:	learn: 0.2352210	total: 30.8s	remaining: 2m 26s
174:	learn: 0.2349614	total: 30.9s	remaining: 2m 25s
175:	learn: 0.2346797	total: 31.1s	remaining: 2m 25s
176:	learn: 0.2342475	total: 31.3s	remaining: 2m

315:	learn: 0.2107108	total: 54.6s	remaining: 1m 58s
316:	learn: 0.2106113	total: 54.7s	remaining: 1m 57s
317:	learn: 0.2104499	total: 54.9s	remaining: 1m 57s
318:	learn: 0.2103636	total: 55s	remaining: 1m 57s
319:	learn: 0.2102518	total: 55.2s	remaining: 1m 57s
320:	learn: 0.2101271	total: 55.3s	remaining: 1m 57s
321:	learn: 0.2100487	total: 55.5s	remaining: 1m 56s
322:	learn: 0.2099747	total: 55.6s	remaining: 1m 56s
323:	learn: 0.2098950	total: 55.8s	remaining: 1m 56s
324:	learn: 0.2097830	total: 55.9s	remaining: 1m 56s
325:	learn: 0.2096928	total: 56.1s	remaining: 1m 55s
326:	learn: 0.2095168	total: 56.2s	remaining: 1m 55s
327:	learn: 0.2094461	total: 56.4s	remaining: 1m 55s
328:	learn: 0.2093819	total: 56.5s	remaining: 1m 55s
329:	learn: 0.2093365	total: 56.7s	remaining: 1m 55s
330:	learn: 0.2092878	total: 56.8s	remaining: 1m 54s
331:	learn: 0.2092254	total: 57s	remaining: 1m 54s
332:	learn: 0.2091476	total: 57.1s	remaining: 1m 54s
333:	learn: 0.2090525	total: 57.3s	remaining: 1m 5

471:	learn: 0.1983574	total: 1m 18s	remaining: 1m 27s
472:	learn: 0.1982435	total: 1m 18s	remaining: 1m 27s
473:	learn: 0.1982071	total: 1m 18s	remaining: 1m 27s
474:	learn: 0.1981728	total: 1m 18s	remaining: 1m 26s
475:	learn: 0.1980969	total: 1m 18s	remaining: 1m 26s
476:	learn: 0.1980261	total: 1m 18s	remaining: 1m 26s
477:	learn: 0.1979154	total: 1m 19s	remaining: 1m 26s
478:	learn: 0.1978567	total: 1m 19s	remaining: 1m 26s
479:	learn: 0.1977830	total: 1m 19s	remaining: 1m 26s
480:	learn: 0.1976314	total: 1m 19s	remaining: 1m 25s
481:	learn: 0.1975312	total: 1m 19s	remaining: 1m 25s
482:	learn: 0.1974426	total: 1m 19s	remaining: 1m 25s
483:	learn: 0.1973884	total: 1m 20s	remaining: 1m 25s
484:	learn: 0.1973369	total: 1m 20s	remaining: 1m 25s
485:	learn: 0.1972955	total: 1m 20s	remaining: 1m 24s
486:	learn: 0.1972368	total: 1m 20s	remaining: 1m 24s
487:	learn: 0.1971713	total: 1m 20s	remaining: 1m 24s
488:	learn: 0.1971176	total: 1m 20s	remaining: 1m 24s
489:	learn: 0.1970340	total:

625:	learn: 0.1901358	total: 1m 43s	remaining: 1m 1s
626:	learn: 0.1900815	total: 1m 43s	remaining: 1m 1s
627:	learn: 0.1900262	total: 1m 43s	remaining: 1m 1s
628:	learn: 0.1899556	total: 1m 43s	remaining: 1m 1s
629:	learn: 0.1898745	total: 1m 43s	remaining: 1m 1s
630:	learn: 0.1898320	total: 1m 44s	remaining: 1m
631:	learn: 0.1898116	total: 1m 44s	remaining: 1m
632:	learn: 0.1897792	total: 1m 44s	remaining: 1m
633:	learn: 0.1897482	total: 1m 44s	remaining: 1m
634:	learn: 0.1897137	total: 1m 44s	remaining: 1m
635:	learn: 0.1896340	total: 1m 45s	remaining: 1m
636:	learn: 0.1896032	total: 1m 45s	remaining: 59.9s
637:	learn: 0.1895546	total: 1m 45s	remaining: 59.8s
638:	learn: 0.1894820	total: 1m 45s	remaining: 59.6s
639:	learn: 0.1894558	total: 1m 45s	remaining: 59.5s
640:	learn: 0.1894209	total: 1m 45s	remaining: 59.3s
641:	learn: 0.1894020	total: 1m 46s	remaining: 59.1s
642:	learn: 0.1893808	total: 1m 46s	remaining: 58.9s
643:	learn: 0.1892955	total: 1m 46s	remaining: 58.8s
644:	learn:

783:	learn: 0.1842292	total: 2m 10s	remaining: 36.1s
784:	learn: 0.1841772	total: 2m 11s	remaining: 35.9s
785:	learn: 0.1841545	total: 2m 11s	remaining: 35.8s
786:	learn: 0.1841380	total: 2m 11s	remaining: 35.6s
787:	learn: 0.1841172	total: 2m 11s	remaining: 35.4s
788:	learn: 0.1840946	total: 2m 11s	remaining: 35.3s
789:	learn: 0.1840713	total: 2m 12s	remaining: 35.1s
790:	learn: 0.1840615	total: 2m 12s	remaining: 34.9s
791:	learn: 0.1840016	total: 2m 12s	remaining: 34.8s
792:	learn: 0.1839747	total: 2m 12s	remaining: 34.6s
793:	learn: 0.1839594	total: 2m 12s	remaining: 34.4s
794:	learn: 0.1839204	total: 2m 12s	remaining: 34.2s
795:	learn: 0.1838794	total: 2m 12s	remaining: 34.1s
796:	learn: 0.1838510	total: 2m 13s	remaining: 33.9s
797:	learn: 0.1837857	total: 2m 13s	remaining: 33.7s
798:	learn: 0.1837431	total: 2m 13s	remaining: 33.6s
799:	learn: 0.1837078	total: 2m 13s	remaining: 33.4s
800:	learn: 0.1836688	total: 2m 13s	remaining: 33.2s
801:	learn: 0.1836272	total: 2m 14s	remaining:

939:	learn: 0.1794929	total: 2m 38s	remaining: 10.1s
940:	learn: 0.1794800	total: 2m 38s	remaining: 9.96s
941:	learn: 0.1794660	total: 2m 39s	remaining: 9.8s
942:	learn: 0.1794308	total: 2m 39s	remaining: 9.63s
943:	learn: 0.1793953	total: 2m 39s	remaining: 9.46s
944:	learn: 0.1793707	total: 2m 39s	remaining: 9.29s
945:	learn: 0.1793432	total: 2m 39s	remaining: 9.12s
946:	learn: 0.1793025	total: 2m 40s	remaining: 8.95s
947:	learn: 0.1792669	total: 2m 40s	remaining: 8.79s
948:	learn: 0.1792579	total: 2m 40s	remaining: 8.62s
949:	learn: 0.1792534	total: 2m 40s	remaining: 8.45s
950:	learn: 0.1792490	total: 2m 40s	remaining: 8.28s
951:	learn: 0.1792172	total: 2m 40s	remaining: 8.11s
952:	learn: 0.1791752	total: 2m 41s	remaining: 7.95s
953:	learn: 0.1791525	total: 2m 41s	remaining: 7.78s
954:	learn: 0.1790899	total: 2m 41s	remaining: 7.61s
955:	learn: 0.1790773	total: 2m 41s	remaining: 7.45s
956:	learn: 0.1790677	total: 2m 41s	remaining: 7.28s
957:	learn: 0.1790332	total: 2m 42s	remaining: 

In [37]:
company = 'ОАО "МЕСЯГУТОВСКИЙ КСОМ"'
extract_features(company, test=True)

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.05263158, 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.33333333,
        0.22807018, 0.61538462, 0.38461538, 0.        , 1.        ,
        0.        ]),
 array([0.        , 0.05263158, 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.33333333, 0.22807018, 0.61538462,
        0.38461538, 0.        , 1.        , 0.        , 0.66666667,
        0.07017544, 0.75      , 0.25      , 0.        , 0.        ,
        0.        ]),
 array([0.33333333, 0.22807018, 0.61538462, 0.38461538, 0.        ,
        1.        , 0.        , 0.66666667, 0.07017544, 0.75      ,
        0.25      , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ])]

In [19]:
transform(company, clf.predict(extract_features(company, test=True)))

[2.]
[1.]
[2.]


'ОАО "Месягутовский КСОМ"'

In [ ]:
for company in train_companies[200000:200050]:
    print('Original: ' company)
    result.write(transform(company, clf.predict(extract_features(company, test=True))) + '\n')

In [79]:
test_companies[1000:1200]

['КРЕСТЬЯНСКОЕ (ФЕРМЕРСКОЕ) ХОЗЯЙСТВО "ЮПИТЕР" БУРАЕВСКОГО РАЙОНА РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МКУ ОТДЕЛ ОБРАЗОВАНИЯ АМР БР РБ',
 'МОУ НОШ Д.НАРЫШЕВО',
 'МНОШ ДЕРЕВНИ ЯКУНИНО',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "НАЧАЛЬНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д.АИТОВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МУНИЦИПАЛЬНОЕ ДОШКОЛЬНОЕ ОБРАЗОВАТЕЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "ДЕТСКИЙ САД Д.КУЗБАЕВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МОБУ СОШ ИМ.М.Г.ХАЙРУЛЛИНОЙ Д.КАШКАЛЕВО',
 'МОБУ ООШ Д.МАМАДЫ',
 'МООШ Д.КАРГАЛЫ',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ "НАЧАЛЬНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д.АСАВТАМАК" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МОБУ НОШ №3 С.БУРАЕВО',
 'МОБУ СОШ Д. БОЛЬШЕШУКШАНОВО',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "ОСНОВНАЯ ОБЩЕОБРАЗОВАТЕЛЬНАЯ ШКОЛА Д. БОЛЬШЕШУКШАНОВО" МУНИЦИПАЛЬНОГО РАЙОНА БУРАЕВСКИЙ РАЙОН РЕСПУБЛИКИ БАШКОРТОСТАН',
 'МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВА

In [38]:
result = open('result.txt', 'w')
for company in test_companies:
    result.write(transform(company, clf.predict(extract_features(company, test=True))) + '\n')

ValueError: invalid mode: '\w+'